In [1]:
# 加载 tensor
import torch
loaded_pred = torch.load('/home/v-wenxichen/SLAM-LLM/examples/s2s/scripts/debug/s2s_pred.pt', map_location='cuda:0')
loaded_label = torch.load('/home/v-wenxichen/SLAM-LLM/examples/s2s/scripts/debug/s2s_label.pt', map_location='cuda:0')


/tmp/ipykernel_1342153/3611907854.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_pred = torch.load('/home/v-wenxichen/SLAM-LLM/examples/s2s/scripts/debug/s2s_pre

In [2]:
print(loaded_pred)
print(loaded_label.shape)
print(loaded_label)
print(loaded_label.shape)

tensor([[4097,  241, 1057,  ..., 4097, 4097, 4097],
        [4097, 4097, 2031,  ..., 4097, 4097, 4097],
        [4097, 4097, 4097,  ..., 4097, 4097, 4097],
        ...,
        [4097, 4097, 4097,  ..., 4096, 4097, 4097],
        [4097, 4097, 4097,  ..., 2420, 4096, 4097],
        [4097, 4097, 4097,  ..., 3753, 1278, 4096]], device='cuda:0')
torch.Size([7, 253])
tensor([[4097, 1078, 1942,  ..., 4097, 4097, 4097],
        [4097, 4097, 2031,  ..., 4097, 4097, 4097],
        [4097, 4097, 4097,  ..., 4097, 4097, 4097],
        ...,
        [4097, 4097, 4097,  ..., 4096, 4097, 4097],
        [4097, 4097, 4097,  ..., 2420, 4096, 4097],
        [4097, 4097, 4097,  ..., 3753, 1278, 4096]], device='cuda:0')
torch.Size([7, 253])


In [3]:
from slam_llm.utils.snac_utils import reconscruct_snac, reconstruct_tensors
from snac import SNAC
import soundfile as sf

codec_decoder_path = "hubertsiuzdak/snac_24khz"
codec_decoder = SNAC.from_pretrained(codec_decoder_path).eval()
codec_decoder = codec_decoder.cuda()


/home/v-wenxichen/anaconda3/envs/slam/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/v-wenxichen/SLAM-LLM/src/slam_llm/utils/checkpoint_handler.py:18: DeprecationWarning: `torch.distributed._shard.checkpoint` will be deprecated, use `torch.distributed.checkpoint` instead
  from torch.distributed._shard.checkpoint import (
/home/v-wenxichen/anaconda3/envs/slam/lib/python3.10/site-packages/snac/snac.py:106: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_

In [10]:
# audio_outputs = loaded_label
audio_outputs = loaded_pred

In [11]:
# wav_name = "label_s2s"
wav_name = "pred_s2s"
# def reconstruct_audio(audio_outputs):
audio_tokens = [audio_outputs[layer] for layer in range(7)]
audiolist = reconscruct_snac(audio_tokens)

In [12]:
audiolist = audiolist[:-8]

In [13]:
audio = reconstruct_tensors(audiolist)
with torch.inference_mode():
    audio_hat = codec_decoder.decode(audio)

sf.write(f"{wav_name}.wav", audio_hat.squeeze().cpu().numpy(), 24000)